Research paper used:
https://arxiv.org/pdf/1609.03499.pdf

In [3]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

torch.manual_seed(42); # for reproducibility

In [4]:
words = open('names.txt', 'r').read().splitlines()
chars = sorted(list(set(".".join(words))))
stoi = {c:i for i,c in enumerate(chars)}
itos = {c:i for i,c in stoi.items()}
vocab_size = len(chars)

In [5]:
# preparing the dataset
block_size = 8 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 8]) torch.Size([182625])
torch.Size([22655, 8]) torch.Size([22655])
torch.Size([22866, 8]) torch.Size([22866])


In [6]:
# validate the dataset
for x,y in zip(Xtr[:20],Ytr[:20]):
  print(''.join([itos[i.item()] for i in x]), '-->' ,itos[y.item()])

........ --> y
.......y --> u
......yu --> h
.....yuh --> e
....yuhe --> n
...yuhen --> g
..yuheng --> .
........ --> d
.......d --> i
......di --> o
.....dio --> n
....dion --> d
...diond --> r
..diondr --> e
.diondre --> .
........ --> x
.......x --> a
......xa --> v
.....xav --> i
....xavi --> e


In [224]:
# Need 3 methods, init (obviously), call and parameters (to set we 'need grads')

# -----------------------------------------------------------------------------------------------
class Linear:
  def __init__(self, fan_in, fan_out, bias=True):
    self.weight = torch.randn((fan_in, fan_out)) / fan_in**0.5 # note: kaiming init
    self.bias = torch.zeros(fan_out) if bias else None
  
  def __call__(self, x):
    self.out = x @ self.weight
    if self.bias is not None:
      self.out += self.bias
    return self.out
  
  def parameters(self):
    return [self.weight] + ([] if self.bias is None else [self.bias])

# -----------------------------------------------------------------------------------------------
class BatchNorm1d:
  def __init__(self, dim, momentum=0.1, epsilon=1e-5):
    pass

# -----------------------------------------------------------------------------------------------
class Tanh:
  def __call__(self, x):
    self.out = torch.tanh(x)
    return self.out
  def parameters(self):
    return []

# -----------------------------------------------------------------------------------------------
class Embedding:
  def __init__(self, num_embeddings, embedding_dim):
    self.weight = torch.randn((num_embeddings, embedding_dim))

  def __call__(self, idx):
    self.out = self.weight[idx]
    return self.out
  
  def parameters(self):
    return [self.weight]

# -----------------------------------------------------------------------------------------------
class Flatten: # this matches pytorch's nn.Flatten API
  def __call__(self, x):
    self.out = x.view(x.size(0), -1)
    return self.out

  def parameters(self):
    return []

# -----------------------------------------------------------------------------------------------
class FlattenConsecutive: # this is better!
  def __init__(self, n):
    self.n = n

  def __call__(self, x):
    B, T, C = x.shape
    x = x.view(B, T//self.n, C*self.n)
    if x.shape[1] == 1:
      x = x.squeeze(1)
    self.out = x
    return self.out

  def parameters(self):
    return []

# -----------------------------------------------------------------------------------------------
class Sequential:
  def __init__(self, layers):
    self.layers = layers

  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    self.out = x
    return self.out

  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]

In [266]:
# Boiler plate of model initialization
# model = Sequential([
#   Embedding(), # embedding
#   Flatten(), Linear(), BatchNorm1d(), Tanh(), # flat -> input -> batchnorm -> nonlinearity
#   Linear(), # output layer
# ])


# Initialize the model
emb_sz = 10
n_hidden = 200

model = Sequential([
  Embedding(vocab_size, emb_sz), # B, S, C
  FlattenConsecutive(2), Linear(emb_sz  *2, n_hidden), Tanh(),
  FlattenConsecutive(2), Linear(n_hidden*2, n_hidden), Tanh(),
  FlattenConsecutive(2), Linear(n_hidden*2, n_hidden), Tanh(),
  Linear(n_hidden, vocab_size)
])

parameters = model.parameters()
print(sum([p.nelement() for p in parameters]))
for p in parameters:
  p.requires_grad = True

170297


In [264]:
Xbs = Xtr[range(32)]
model(Xbs).shape

torch.Size([32, 27])

In [256]:
# training loop

In [10]:
# plot loss

In [11]:
# cal split loss

In [12]:
# sampling

In [ ]:
# rewatch the last part!!